In [ ]:
import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import h5py
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

In [ ]:
vmec_file = "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4"
helias5 = jsb.flux_surfaces.FluxSurface.from_hdf5(vmec_file)

In [ ]:

ss, tt, pp = jnp.meshgrid(jnp.linspace(0, 1, 20), jnp.linspace(0, 2 * jnp.pi, 30), jnp.linspace(0, 2 * jnp.pi /10, 40), indexing='ij')

f_normal = helias5.position(ss, tt, pp)


In [ ]:
fnp2 = np.array(helias5.cartesian_position(ss, tt, pp)).reshape(-1, 3)

In [ ]:
import numpy as np
fx  = f_normal[..., 0]  * np.cos(f_normal[..., 2])
fy  = f_normal[..., 0]  * np.sin(f_normal[..., 2])
fz  = f_normal[..., 1]

fnp = np.stack([fx, fy, fz], axis=-1).reshape(-1, 3)

In [ ]:
import pyvista as pv

In [ ]:
plotter = pv.Plotter()  
plotter.add_mesh(pv.PolyData(fnp2), color='lightblue', show_edges=True, opacity=0.5)
plotter.show()

In [ ]:
import StellBlanket.SBGeom as SBGeom
import numpy as np

fs_old = SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file)

s_new = np.array(s *  jnp.ones_like(tt).ravel())
d_new = np.array(jnp.zeros_like(tt).ravel())
p_new = np.array(pp.ravel())
t_new = np.array(tt.ravel())

f_new = fs_old.Return_Position(s_new, d_new, t_new, p_new).reshape(tt.shape + (3,))

f_new_R = jnp.sqrt(f_new[..., 0]**2 + f_new[..., 1]**2)
f_new_Z = f_new[..., 2]
f_new_phi = (jnp.arctan2(f_new[..., 1], f_new[..., 0]) + 2 * jnp.pi) % (2 *jnp.pi)

f_total_new = jnp.stack([f_new_R, f_new_Z, f_new_phi], axis=-1)

print(f_normal - f_total_new)